<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Linear-Regression" data-toc-modified-id="Linear-Regression-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Linear Regression</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

In [127]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import warnings
import sklearn
from pymystem3 import Mystem
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import warnings
warnings.filterwarnings("ignore")

import optuna

from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error

from tqdm import notebook

In [128]:
factory_2019 = pd.read_csv('C://Users/Admin///Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2019 почасовки.csv', sep=';')
factory_2020 = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2020 почасовки.csv', sep=';')
factory_2021 = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2021 почасовки.csv', sep=';')
factory_2022 = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2022 почасовки.csv', sep=';')

In [129]:
weather = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Погодные_данные/weather_new.csv', sep=';', 
                      on_bad_lines='skip', decimal=",")

In [130]:
factory = factory_2019.append(factory_2020).append(factory_2021).append(factory_2022).reset_index(drop=True)

In [131]:
factory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29928 entries, 0 to 29927
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       29928 non-null  object
 1   2       29928 non-null  object
 2   3       29928 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 701.6+ KB


In [132]:
factory.describe()

,3
count,29928.000000
mean,45171.418170
std,7777.700706
min,5967.000000
25%,40602.750000
50%,45377.000000
75%,50136.500000
max,78343.000000


In [133]:
factory['datetime'] = factory['1']+' '+factory['2']

In [134]:
factory['datetime'] = pd.to_datetime(factory['datetime'], format='%d.%m.%Y %H:%M:%S')

In [135]:
factory = factory.rename(columns = {'3':'expenditure'})

In [136]:
factory = factory[['datetime', 'expenditure']]

In [137]:
factory.head()

,datetime,expenditure
0,2019-01-01 00:00:00,18068
1,2019-01-01 01:00:00,18139
2,2019-01-01 02:00:00,18171
3,2019-01-01 03:00:00,18001
4,2019-01-01 04:00:00,18119


In [138]:
factory = factory.set_index('datetime')

In [139]:
def cloud(a):
    a = str(a)
    a = a.replace('%', '')
    a = a.replace('.', '')
    if a == 'Облаков нет':
        return 0
    elif a =='Небо не видно из-за тумана и/или других метеорологических явлений':
        return 100
    elif '–' or ' ' in a:
        return a[:3]
    else:
        return a
    
def delited_1(a):
    a=str(a)
    a=a.replace('–', '')
    a=a.replace(' ', '')
    return a

def fill(a):
    if a == 'nan':
        return 100
    else:
        return a

In [140]:
weather = weather[['Местное время в Самаре','T']] #, 'U', 'Ff', 'N'
weather = weather.rename(columns = {'Местное время в Самаре':'datetime'})
weather['datetime'] = pd.to_datetime(weather['datetime'], format='%d.%m.%Y %H:%M')
weather = weather.set_index('datetime')
#weather['N'] = weather['N'].apply(cloud).apply(delited_1).apply(fill).astype('int')
weather = weather.fillna(method='bfill')
weather = weather.resample('1H').median()
weather = weather.reset_index()
for i in range(len(weather)-2):
    if weather['T'][i]==0:
        continue
    elif weather['T'][i]==len(weather)-1:
        break
    elif weather['T'][i]!='NaN':
        continue
    elif weather['T'][i]=='NaN' and weather['T'][i+2]=='NaN':
        weather['T'][i]= weather['T'][i-1]
    elif weather['T'][i]=='NaN' and weather['T'][i+1] =='NaN':
        weather['T'][i]= (weather['T'][i-1]+weather['T'][i+2])/2
    elif weather['T'][i]=='NaN' and weather['T'][i+1]!='NaN':
        weather['T'][i]= (weather['T'][i-1] +weather['T'][i+1])/2
weather = weather.set_index('datetime')
weather = weather.fillna(method='bfill')

In [141]:
from scipy import stats
from scipy.stats import kurtosis
from scipy.signal import find_peaks
data = factory.join(weather, on='datetime')
def make_features(data, max_lag, rolling_mean_size):
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data['is_weekend'] = data.dayofweek.isin([5,6])*1
    data['hour'] = data.index.hour
    data['expanding_window'] = data['expenditure'].expanding(3).mean()
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['expenditure'].shift(lag)
    data['rolling_mean'] = data['expenditure'].shift().rolling(rolling_mean_size).mean()
    data['kurtosis'] = data['rolling_mean'].shift().rolling(7).apply(lambda x: kurtosis(x))
    data['peaks'] = data['rolling_mean'].shift().rolling(7).apply(lambda x: len(find_peaks(x)[0]))

In [142]:
make_features(data, 24, 24)
data = data.dropna()

In [143]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29897 entries, 2019-01-02 07:00:00 to 2022-05-31 23:00:00
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   expenditure       29897 non-null  int64  
 1   T                 29897 non-null  float64
 2   month             29897 non-null  int64  
 3   day               29897 non-null  int64  
 4   dayofweek         29897 non-null  int64  
 5   is_weekend        29897 non-null  int32  
 6   hour              29897 non-null  int64  
 7   expanding_window  29897 non-null  float64
 8   lag_1             29897 non-null  float64
 9   lag_2             29897 non-null  float64
 10  lag_3             29897 non-null  float64
 11  lag_4             29897 non-null  float64
 12  lag_5             29897 non-null  float64
 13  lag_6             29897 non-null  float64
 14  lag_7             29897 non-null  float64
 15  lag_8             29897 non-null  float64
 16  lag_9

In [150]:
features = data.drop(['expenditure'], axis=1)
target = data['expenditure'] 

# cat_features = ['month', 'day', 'dayofweek', 'is_weekend', 'hour']
# for i in notebook.tqdm(cat_features):
#     features[i] = features[i].astype('str')
# features = pd.get_dummies(features, drop_first=True)

In [151]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, shuffle=False)

In [152]:
features_train

,T,month,day,dayofweek,is_weekend,hour,expanding_window,lag_1,lag_2,lag_3,...,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,rolling_mean,kurtosis,peaks
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-02 07:00:00,-5.9,1,2,2,0,7,18087.968750,17710.0,17963.0,19215.0,...,18028.0,17956.0,16875.0,16872.0,17077.0,17394.0,17834.0,18052.375000,-1.272684,1.0
2019-01-02 08:00:00,-5.8,1,2,2,0,8,18075.151515,18171.0,17710.0,17963.0,...,17820.0,18028.0,17956.0,16875.0,16872.0,17077.0,17394.0,18066.416667,-1.110023,1.0
2019-01-02 09:00:00,-5.8,1,2,2,0,9,18047.294118,17665.0,18171.0,17710.0,...,17767.0,17820.0,18028.0,17956.0,16875.0,16872.0,17077.0,18077.708333,-0.924237,1.0
2019-01-02 10:00:00,-5.8,1,2,2,0,10,18017.685714,17128.0,17665.0,18171.0,...,17522.0,17767.0,17820.0,18028.0,17956.0,16875.0,16872.0,18079.833333,-0.670552,1.0
2019-01-02 11:00:00,-5.0,1,2,2,0,11,17991.416667,17011.0,17128.0,17665.0,...,17464.0,17522.0,17767.0,17820.0,18028.0,17956.0,16875.0,18085.625000,-0.779020,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-27 05:00:00,-12.6,1,27,3,0,5,44841.569206,50386.0,50874.0,48530.0,...,45917.0,45927.0,49353.0,51184.0,54240.0,54105.0,58156.0,51698.750000,-1.500482,0.0
2022-01-27 06:00:00,-12.6,1,27,3,0,6,44841.745127,51995.0,50386.0,50874.0,...,47481.0,45917.0,45927.0,49353.0,51184.0,54240.0,54105.0,51442.041667,-1.587752,0.0
2022-01-27 07:00:00,-12.6,1,27,3,0,7,44842.051233,49580.0,51995.0,50386.0,...,50464.0,47481.0,45917.0,45927.0,49353.0,51184.0,54240.0,51253.500000,-1.109482,0.0


In [153]:
# numeric = ['T', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 
#            'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 
#            'lag_13', 'lag_14', 'lag_15', 'lag_16', 'lag_17', 'lag_18', 'lag_19', 
#            'lag_20', 'lag_21', 'lag_22', 'lag_23', 'lag_24', 'rolling_mean', 'kurtosis', 'peaks', 'expanding_window']
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

In [154]:
features.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29897 entries, 2019-01-02 07:00:00 to 2022-05-31 23:00:00
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   T                 29897 non-null  float64
 1   month             29897 non-null  int64  
 2   day               29897 non-null  int64  
 3   dayofweek         29897 non-null  int64  
 4   is_weekend        29897 non-null  int32  
 5   hour              29897 non-null  int64  
 6   expanding_window  29897 non-null  float64
 7   lag_1             29897 non-null  float64
 8   lag_2             29897 non-null  float64
 9   lag_3             29897 non-null  float64
 10  lag_4             29897 non-null  float64
 11  lag_5             29897 non-null  float64
 12  lag_6             29897 non-null  float64
 13  lag_7             29897 non-null  float64
 14  lag_8             29897 non-null  float64
 15  lag_9             29897 non-null  float64
 16  lag_1

## Linear Regression

In [155]:
model_LR = LinearRegression()
scores_LR = cross_val_score(model_LR, features_train, target_train, scoring='neg_mean_squared_error', 
                                          cv=TimeSeriesSplit(n_splits=5))
print(scores_LR)
print((scores_LR.mean()*(-1))**0.5)

[-14448335.96123475 -12703595.99864309 -11772481.44831283
 -13390523.10302926 -12600562.25101011]
3603.2068706148425


In [156]:
model_LR.fit(features_test, target_test)

LinearRegression()

## Тестирование 

In [157]:
p = model_LR.predict(features_test)
mean_absolute_percentage_error(p, target_test)

0.056742875149769494

In [158]:
factory.tail()

,expenditure
datetime,
2022-05-31 19:00:00,29406
2022-05-31 20:00:00,30868
2022-05-31 21:00:00,31136
2022-05-31 22:00:00,31731
2022-05-31 23:00:00,30740


In [159]:
data_new = weather.join(factory, on='datetime')
make_features(data_new, 24, 24)
time_for_predict = data_new['2022-06-01 00:00:00':'2022-06-03 23:00:00'].index.to_list()

In [160]:
model = LinearRegression()
model.fit(features, target)

LinearRegression()

In [161]:
for i in notebook.tqdm(time_for_predict):
    target_a = data_new.expenditure[i]
    features_a = data_new[str(i):str(i)].drop(['expenditure'], axis=1)
    prediction = model.predict(features_a)
    data_new.expenditure[str(i)] = (float(prediction))
    data_new = data_new[['T', 'expenditure']]
    make_features(data_new, 24, 24)

  0%|          | 0/72 [00:00<?, ?it/s]

In [162]:
prediction_test = data_new.expenditure['2022-06-01 00:00:00':] #выделяем диапазон предсказанный
prediction_test = prediction_test.reset_index()
prediction_test.head()

,datetime,expenditure
0,2022-06-01 00:00:00,33428.810615
1,2022-06-01 01:00:00,35474.608571
2,2022-06-01 02:00:00,36680.100863
3,2022-06-01 03:00:00,37463.946397
4,2022-06-01 04:00:00,38023.115226


In [163]:
test = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Предприятие_2//test_2.csv', sep=';')
test['prediction'] = prediction_test['expenditure']
mean_absolute_percentage_error(test['prediction'], test['fact'])

0.1113318759494337

In [164]:
mean_squared_error(test['prediction'], test['fact'], squared=False)

5441.014599611397

## Вывод

Линейная регрессия показала результат лучше, чем бустинг